**Introduction**

This notebook deals with classification of satellite images of roofs into four categories, based on orientation of the roofs.
1. North-South Orientation
2. East-West Orientation
3. Flat Roof
4. Other

The problem was part of Phase 1 of the [Data Science Games Competition][1], held from June 17 to July 10, in which I participated in a team of 4, along with Sue Yang, Sasanka Gandavarapu, and Jim Chen (all students in MIDS Berkeley).
[1]: https://inclass.kaggle.com/c/data-science-game-2016-online-selection

This notebook includes mostly final used methods. A separate notebook with unused scratch work and methods is available on request.

-----------------------------------------------------------------------------------------------------------------------
Below are the necessary package imports and data processing functions. 

In [1]:
import csv
import os
import cv2
import math
import time
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.grid_search import GridSearchCV

from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.filters import gaussian

from sklearn.ensemble import RandomForestClassifier 

There are two different feature sets with which I will be working. 

*First, the raw pixel matrix. 
*Second, a feature set consisting of the height-to-width ratio of the image, and a color histogram. 

All load methods (except for the test set) add the image itself + 3 rotations of the image to the feature set (90, 180, and 270 degree rotations). Methods described below:

1. extract_color_histogram: Given an image, returns a feature vector of length 512 representing the color distribution of the image.
2. load_unlabelled_pixels: Given a list of image IDs, create a numpy (flattened) pixel matrix for each. Used on unlabelled images.
3. load_unlabelled_colors: Given a list of image IDs, create a numpy color historgram feature vector for each. Used on unlabelled images.
4. load_train_cv: Returns a train and test set of pixel matrices from the list of labelled examples in id_train.csv. 
5. load_train_features: Returns a train and test set of color histograms feature vectors from a list of labelled examples in id_train.csv.
6. load_test: Returns the pixel matrices feature array corresponding to the test set for the competition, given in sample_submission4.csv
7. load_test_hist: Returns the color histogram feature array corresponding to the test set for the competition, given in sample_submission4.csv

In [2]:

'''
Loading data functions
'''
PIXELS = 32
imageSize = PIXELS * PIXELS
num_features = imageSize 

def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
        [0, 180, 0, 256, 0, 256])

    cv2.normalize(hist, hist)

    # return the flattened histogram as the feature vector
    return hist.flatten()

def load_unlabelled_pixels(list_nums):
    X = []
    for num in list_nums:
        file_name = os.path.join('input', str(num) + '.jpg')
        img = cv2.imread(file_name,0)
        img = cv2.resize(img, (PIXELS, PIXELS))
        M = cv2.getRotationMatrix2D((PIXELS/2, PIXELS/2),90,1)
        img90 = cv2.warpAffine(img, M, (PIXELS, PIXELS))
        img180 = cv2.warpAffine(img90, M, (PIXELS, PIXELS))
        img270 = cv2.warpAffine(img180, M, (PIXELS, PIXELS))
        img = np.reshape(img, (1, num_features))
        img90 = np.reshape(img90, (1, num_features))
        img180 = np.reshape(img180, (1, num_features))
        img270 = np.reshape(img270, (1, num_features))
        X.append(img)
        X.append(img90)
        X.append(img180)
        X.append(img270)
    
    X = np.array(X)
    X = X.reshape(X.shape[0], num_features).astype('float32') / 255
    return X

def load_unlabelled_colors(list_nums):
    X = []
    for num in list_nums:
        features = []
        features90 = []
        features180 = []
        features270 = []
        file_name = os.path.join('input', str(num) + '.jpg')
        img = cv2.imread(file_name)
        h, w, blah = img.shape
        ratio = h/float(w)
            
        M = cv2.getRotationMatrix2D((w/2, h/2),90,1)
        img90 = cv2.warpAffine(img, M, (w, h))
        img180 = cv2.warpAffine(img90, M, (w, h))
        img270 = cv2.warpAffine(img180, M, (w, h))
        
        features.extend(extract_color_histogram(img))
        features.append(ratio)
        features90.extend(extract_color_histogram(img90))
        features90.append(1.0/ratio)
        features180.extend(extract_color_histogram(img180))
        features180.append(ratio)
        features270.extend(extract_color_histogram(img270))
        features270.append(1.0/ratio)
            
        X.append(np.array(features))
        X.append(np.array(features90))
        X.append(np.array(features180))
        X.append(np.array(features270))
    
    X = np.array(X)
    return X

def load_train_cv(encoder):
    X_train = []
    y_train = []
    print('Read train images')
    #Read train ids
    
    with open('id_train.csv', 'rb') as csvfile:
        trainreader = csv.reader(csvfile, delimiter=',')
        next(trainreader)
        for row in trainreader:
            #print(row[0])
            file_name = os.path.join('input', row[0] + '.jpg')
            f_nums.remove(int(row[0]))
            img = cv2.imread(file_name,0)
            
            img = cv2.resize(img, (PIXELS, PIXELS))
            M = cv2.getRotationMatrix2D((PIXELS/2, PIXELS/2),90,1)
            img90 = cv2.warpAffine(img, M, (PIXELS, PIXELS))
            img180 = cv2.warpAffine(img90, M, (PIXELS, PIXELS))
            img270 = cv2.warpAffine(img180, M, (PIXELS, PIXELS))
            img = np.reshape(img, (1, num_features))
            img90 = np.reshape(img90, (1, num_features))
            img180 = np.reshape(img180, (1, num_features))
            img270 = np.reshape(img270, (1, num_features))
            X_train.append(img)
            X_train.append(img90)
            X_train.append(img180)
            X_train.append(img270)
            y_train.append(row[1])
            y_train.append(row[1])
            if row[1] == '1':
                y_train.append ('2')
                y_train.append ('2')
            elif row[1] == '2':
                y_train.append ('1')
                y_train.append ('1')
            else:
                y_train.append(row[1])
                y_train.append(row[1])
                
    #print X_train.shape
    X_train = np.array(X_train)
    y_train = np.array(y_train).astype('int32')

    #y_train = encoder.fit_transform(y_train).astype('int32')

    X_train, y_train = shuffle(X_train, y_train, random_state=42)

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3,random_state=42)

    X_train = X_train.reshape(X_train.shape[0], num_features).astype('float32') / 255.
    X_test = X_test.reshape(X_test.shape[0], num_features).astype('float32') / 255.
    #X_train = X_train.astype('float32') / 255.
    #X_test = X_test.astype('float32') / 255.

    return X_train, y_train, X_test, y_test, encoder

def load_train_features(encoder):
    X_train = []
    y_train = []
    print('Read train images')
    #Read train ids
    
    with open('id_train.csv', 'rb') as csvfile:
        trainreader = csv.reader(csvfile, delimiter=',')
        next(trainreader)
        for row in trainreader:
            #print(row[0])
            features = []
            features90 = []
            features180 = []
            features270 = []
            file_name = os.path.join('input', row[0] + '.jpg')
            img = cv2.imread(file_name)
            h, w, blah = img.shape
            ratio = h/float(w)
            
            M = cv2.getRotationMatrix2D((w/2, h/2),90,1)
            img90 = cv2.warpAffine(img, M, (w, h))
            img180 = cv2.warpAffine(img90, M, (w, h))
            img270 = cv2.warpAffine(img180, M, (w, h))
            
            features.extend(extract_color_histogram(img))
            features.append(ratio)
            features90.extend(extract_color_histogram(img90))
            features90.append(1.0/ratio)
            features180.extend(extract_color_histogram(img180))
            features180.append(ratio)
            features270.extend(extract_color_histogram(img270))
            features270.append(1.0/ratio)
            
            X_train.append(np.array(features))
            X_train.append(np.array(features90))
            X_train.append(np.array(features180))
            X_train.append(np.array(features270))
            y_train.append(row[1])
            y_train.append(row[1])
            if row[1] == '1':
                y_train.append ('2')
                y_train.append ('2')
            elif row[1] == '2':
                y_train.append ('1')
                y_train.append ('1')
            else:
                y_train.append(row[1])
                y_train.append(row[1])
                
    X_train = np.array(X_train)
    y_train = np.array(y_train).astype('int32')

    X_train, y_train = shuffle(X_train, y_train, random_state=42)

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

    return X_train, y_train, X_test, y_test, encoder

def load_test():
    print('Read test images')
    X_test = []
    X_test_id=[]
    with open('sample_submission4.csv', 'rb') as csvfile:
        testreader = csv.reader(csvfile, delimiter=',')
        next(testreader)
        for row in testreader:
            #print('Load folder c{}'.format(j))
            file_name = os.path.join('input', row[0] + '.jpg')
            f_nums.remove(int(row[0]))
            img = cv2.imread(file_name,0)
            img = cv2.resize(img, (PIXELS, PIXELS))
            #img = img.transpose(2, 0, 1)
            img = np.reshape(img, (1, num_features))
            X_test.append(img)
            X_test_id.append(row[0])

    X_test = np.array(X_test)
    X_test_id = np.array(X_test_id)

    X_test = X_test.reshape(X_test.shape[0],num_features ).astype('float32') / 255.

    return X_test, X_test_id

def load_test_hist():
    print('Read test images')
    X_test = []
    X_test_id=[]
    with open('sample_submission4.csv', 'rb') as csvfile:
        testreader = csv.reader(csvfile, delimiter=',')
        next(testreader)
        for row in testreader:
            #print('Load folder c{}'.format(j))
            features = []
            file_name = os.path.join('input', row[0] + '.jpg')
            img = cv2.imread(file_name)
            h, w, blah = img.shape
            ratio = h/float(w)
            features.extend(extract_color_histogram(img))
            features.append(ratio)
            X_test.append(np.array(features))
            X_test_id.append(row[0])
    X_test = np.array(X_test)
    X_test_id = np.array(X_test_id)
    return X_test, X_test_id


In [ ]:
'''
load training data, test, and unlabelled data. Cell takes ~3-5 mins to run
'''
#Creating list of file IDs (so we can eliminate the labelled and test examples to get the list of unlabelled examples)
f_nums = []
for root, dirnames, filenames in os.walk('input'):
    for f in filenames:
        f_nums.append(int(f[:-4]))    

encoder = LabelEncoder()

# load the training and validation data sets for first feature set
train_X, train_y, valid_X, valid_y, encoder = load_train_cv(encoder)

# load the training and validation data sets for second feature set
train_hist_X, train_hist_y, valid_hist_X, valid_hist_y, encoder = load_train_features(encoder)

print "Dimensions of training and label set for feature set 1: Raw Pixels"
print('Train set shape:', train_X.shape, 'Dev (valid) set shape:', valid_X.shape, valid_X.dtype)
print('Train labels shape:', train_y.shape, 'Dev (valid) labels shape:', valid_y.shape)

print "Dimensions of training and label set for feature set 1: Raw Pixels"
print('Train set shape:', train_hist_X.shape, 'Dev (valid) set shape:', valid_hist_X.shape, valid_hist_X.dtype)
print('Train labels shape:', train_hist_y.shape, 'Dev (valid) labels shape:', valid_hist_y.shape)

# loading test data
X_test, X_test_id = load_test()
X_test_hist, X_test_id_hist = load_test_hist()
print('Test shape:', X_test.shape, ' Test hist shape: ', X_test_hist.shape)

print 'Making sure test sets for both feature sets are in the same order:'
check = 'All good'
for a, b in zip(X_test_id, X_test_id_hist):
    if a != b:
        check = 'Not equal'
print check

print 'Loading unlabelled data:'
X_unlabelled_pixels = load_unlabelled_pixels(f_nums)
X_unlabelled_colors = load_unlabelled_colors(f_nums)
print('X_unlabelled_pixels shape:', X_unlabelled_pixels.shape, 'X_unlabelled_colors shape:', X_unlabelled_colors.shape)


Read train images


After experimenting with various models (KNN, NB, DecisionTrees, GradientBoostedTrees, AdaBoost etc.), Random Forest performed the best. Below, we find the best parameters for the Random Forest models on both feature vector sets. The 2 cells below take 50-60 mins each to run

In [14]:
# Pick n_estimators=80
#Lets tune for max_depth and min_samples_split
param_test = {'n_estimators':range(50,201,50), 'bootstrap':[True, False], 
              'class_weight':['balanced', 'balanced_subsample', None] }
clf = RandomForestClassifier(random_state=42)
gsearch2 = GridSearchCV(estimator = clf, param_grid = param_test, n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_hist_X,train_hist_y)
print 'Best parameter set: ', gsearch2.best_params_
print 'Best score: ', gsearch2.best_score_

Best parameter set:  {'n_estimators': 200, 'bootstrap': False, 'class_weight': 'balanced'}
Best score:  0.564953920884


In [15]:
# Pick n_estimators=80
#Lets tune for max_depth and min_samples_split
param_test = {'n_estimators':range(50,201,50), 'bootstrap':[True, False], 
              'class_weight':['balanced', 'balanced_subsample', None] }
clf = RandomForestClassifier(random_state=42)
gsearch2 = GridSearchCV(estimator = clf, param_grid = param_test, n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_X,train_y)
print gsearch2.best_params_
print gsearch2.best_score_

[mean: 0.59317, std: 0.00503, params: {'n_estimators': 50, 'bootstrap': True, 'class_weight': 'balanced'}, mean: 0.59960, std: 0.00598, params: {'n_estimators': 100, 'bootstrap': True, 'class_weight': 'balanced'}, mean: 0.60071, std: 0.00660, params: {'n_estimators': 150, 'bootstrap': True, 'class_weight': 'balanced'}, mean: 0.60049, std: 0.00715, params: {'n_estimators': 200, 'bootstrap': True, 'class_weight': 'balanced'}, mean: 0.59094, std: 0.00509, params: {'n_estimators': 50, 'bootstrap': True, 'class_weight': 'balanced_subsample'}, mean: 0.59942, std: 0.00206, params: {'n_estimators': 100, 'bootstrap': True, 'class_weight': 'balanced_subsample'}, mean: 0.59991, std: 0.00472, params: {'n_estimators': 150, 'bootstrap': True, 'class_weight': 'balanced_subsample'}, mean: 0.60022, std: 0.00596, params: {'n_estimators': 200, 'bootstrap': True, 'class_weight': 'balanced_subsample'}, mean: 0.60259, std: 0.00461, params: {'n_estimators': 50, 'bootstrap': True, 'class_weight': None}, mean:

Below I attempt to fit the data separately, with the two Random Forests with the best parameters. Apart from the score, we look at the classification report to see the errors so adjustments can be made accordingly.

In [22]:

rfc2 = RandomForestClassifier(n_estimators=200, bootstrap=False, random_state=42)
rfc2.fit(train_X, train_y)

rfc5 = RandomForestClassifier(n_estimators=200, bootstrap=False, class_weight = 'balanced', random_state=42)
rfc5.fit(train_hist_X, train_hist_y)

print 'Accuracy (a random forest):', rfc2.score(valid_X, valid_y)
print classification_report(rfc2.predict(valid_X),valid_y)

print 'Accuracy (a random forest):', rfc5.score(valid_hist_X, valid_hist_y)
print classification_report(rfc5.predict(valid_hist_X),valid_hist_y)

Accuracy (a random forest): 0.638958333333
             precision    recall  f1-score   support

          1       0.76      0.69      0.72      3572
          2       0.84      0.63      0.72      4276
          3       0.26      0.65      0.37       409
          4       0.33      0.53      0.41      1343

avg / total       0.72      0.64      0.66      9600

Accuracy (a random forest): 0.592291666667
             precision    recall  f1-score   support

          1       0.45      0.42      0.44      3429
          2       0.43      0.44      0.44      3124
          3       0.93      0.91      0.92      1039
          4       0.89      0.95      0.91      2008

avg / total       0.59      0.59      0.59      9600



After trying the two random forest models individually, I noticed that the model on the pixel matrix classified classes 1 and 2 with good accuracy, while the model on the color histogram classified classes 3 and 4 with good accuracy. There is scope to combine the two models to create overall better classification.

I split the training data in half, and train the two individual models on the first half of the train data. Then, I take the prediction probabilities of the models on the second half of the data, and train a new RandomForest Classifier. Then, we can look for this model's accuracy on the dev set. We also create a submission for the leaderboard, of predictions on the test set.

In [27]:
'''
Random Forest
'''

train_X_step1, train_X_step2, train_y_step1, train_y_step2 = train_test_split(train_X, train_y, test_size=0.5, random_state=42)
train_hist_step1, train_hist_step2, train_yhist_step1, train_yhist_step2 = train_test_split(train_hist_X, train_hist_y, 
                                                                                            test_size=0.5, random_state=42)

#checking that training sets are the same for both feature sets
check = 'All good'
for a, b in zip(train_y_step1, train_yhist_step1):
    if a != b:
        check = 'Not equal'
print check, ' step 1'

check = 'All good'
for a, b in zip(train_y_step2, train_yhist_step2):
    if a != b:
        check = 'Not equal'
print check, ' step 2'

#Training 2 initial models
rfc2 = RandomForestClassifier(n_estimators=200, bootstrap=False, random_state=42)
rfc2.fit(train_X_step1, train_y_step1)

rfc5 = RandomForestClassifier(n_estimators=200, bootstrap=False, class_weight = 'balanced', random_state=42)
rfc5.fit(train_hist_step1, train_yhist_step1)

print 'Accuracy (random forest on pixels):', rfc2.score(train_X_step2, train_y_step2)
print classification_report(rfc2.predict(train_X_step2),train_y_step2)

print 'Accuracy (random forest on color histogram):', rfc5.score(train_hist_step2, train_yhist_step2)
print classification_report(rfc5.predict(train_hist_step2),train_yhist_step2)

final_preds12_train = rfc2.predict_proba(train_X_step2)
final_preds34_train = rfc5.predict_proba(train_hist_step2)

final_preds12_valid = rfc2.predict_proba(valid_X)
final_preds34_valid = rfc5.predict_proba(valid_hist_X)        

#Creating training set for final model
train_final = np.array([np.concatenate((final_preds12_train[i],final_preds34_train[i])) for i in range(len(final_preds12_train))])
valid_final = np.array([np.concatenate((final_preds12_valid[i],final_preds34_valid[i])) for i in range(len(final_preds12_valid))])

rfc6 = RandomForestClassifier(n_estimators=100, bootstrap=False)
rfc6.fit(train_final, train_y_step2)
fin_preds = rfc6.predict(valid_final)
print 'Accuracy (combined random forest):', rfc6.score(valid_final, valid_y)
print classification_report(fin_preds,valid_y)

preds1 = rfc2.predict_proba(X_test)
preds2 = rfc5.predict_proba(X_test_hist)

test_final = np.array([np.concatenate((preds1[i], preds2[i])) for i in range(len(preds1))])
preds_leaderboard = rfc6.predict(test_final)

def create_submission(predictions, test_id):
     with open('submit_rf6.csv', 'wb') as mycsvfile:
            thedatawriter = csv.writer(mycsvfile)
            thedatawriter.writerow(['Id','label'])
            for pred,testid  in zip(predictions,test_id):
                out = [testid,pred]
                thedatawriter.writerow(out)
    

print('Creating Submission')
create_submission(preds_leaderboard, X_test_id)
print('Done')

All good  step 1
All good  step 2
Accuracy (random forest on pixels): 0.617767857143
             precision    recall  f1-score   support

          1       0.75      0.66      0.70      4271
          2       0.83      0.62      0.71      5031
          3       0.22      0.57      0.32       463
          4       0.30      0.52      0.38      1435

avg / total       0.71      0.62      0.65     11200

Accuracy (random forest on color histogram): 0.540535714286
             precision    recall  f1-score   support

          1       0.46      0.42      0.44      4169
          2       0.41      0.42      0.42      3679
          3       0.88      0.79      0.83      1313
          4       0.69      0.86      0.76      2039

avg / total       0.54      0.54      0.54     11200

Accuracy (combined random forest): 0.797708333333
             precision    recall  f1-score   support

          1       0.78      0.80      0.79      3119
          2       0.82      0.77      0.80      3438
   

We see above, that the combined model is able to achieve an accuracy of 80%.

Below, we try and incorporate use of the unlabelled images to improve the classifiers by co-training. We train the first two classifiers as above. Then, we label the unlabelled images using the most confident prediction from the two classifiers. Once done, we re-train the original classifiers with the labelled examples and carry out the same exercise once again. Thus, the two models train each other with the labelled examples.

In [32]:
'''
Random Forest
'''
rfc1 = RandomForestClassifier(n_estimators=200, bootstrap=False, random_state=42)
rfc1.fit(train_X, train_y)
rfc3 = RandomForestClassifier(n_estimators=200, bootstrap=False, random_state=42)
rfc3.fit(train_hist_X, train_hist_y)

train_X_pixels = train_X
train_X_colors = train_hist_X
train_y_pixels = train_y
train_y_colors = train_hist_y

for i in range(100):
    pred1 = rfc1.predict(np.array([X_unlabelled_pixels[i]]))
    pred2 = rfc3.predict(np.array([X_unlabelled_colors[i]]))
    
    pred1_prob = rfc1.predict_proba(np.array([X_unlabelled_pixels[i]]))[0][int(pred1[0])-1]
    pred2_prob = rfc3.predict_proba(np.array([X_unlabelled_colors[i]]))[0][int(pred2[0])-1]
    
    np.insert(train_X_pixels, len(train_X_pixels), X_unlabelled_pixels[i])
    np.insert(train_X_colors, len(train_X_colors), X_unlabelled_colors[i])
    if pred1_prob > pred2_prob:
        np.insert(train_y_pixels, len(train_y_pixels), int(pred1[0]))
        np.insert(train_y_colors, len(train_y_colors), int(pred1[0]))
    else:
        np.insert(train_y_pixels, len(train_y_pixels), int(pred2[0]))
        np.insert(train_y_colors, len(train_y_colors), int(pred2[0]))
    rfc1.fit(train_X_pixels, train_y_pixels)
    rfc3.fit(train_X_colors, train_y_colors)

print 'Accuracy (a random forest new model pixels):', rfc1.score(valid_X, valid_y)
print classification_report(rfc1.predict(valid_X),valid_y)
print 'Accuracy (a random forest new model colors):', rfc3.score(valid_hist_X, valid_hist_y)
print classification_report(rfc3.predict(valid_hist_X),valid_hist_y)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Accuracy (a random forest new model pixels): 0.638958333333
             precision    recall  f1-score   support

          1       0.76      0.69      0.72      3572
          2       0.84      0.63      0.72      4276
          3       0.26      0.65      0.37       409
          4       0.33      0.53      0.41      1343

avg / total       0.72      0.64      0.66      9600

Accuracy (a random forest new model colors): 0.585729166667
             precision    recall  f1-score   support

          1       0.46      0.42      0.44      3543
          2       0.41      0.44      0.42      3009
          3       0.93      0.90      0.92      1049
          4       0.87      0.94      0.91      1999

av